In [1]:
!uv pip install -q langchain==0.2.16 langchain-community==0.2.16 langchain-huggingface==0.0.3 langchain-text-splitters==0.2.4 chromadb==0.5.3 langchain-chroma==0.1.3 pandas ipywidgets

In [2]:
!uv pip install -q pyarrow unidecode

In [3]:
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
#from langchain.vectorstores import Chroma
#from langchain_chroma import Chroma
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter
from tqdm import tqdm
import requests
import json
import torch
from langchain.schema import Document 



from langchain.chains import StuffDocumentsChain, RetrievalQA, LLMChain, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.document_loaders import TextLoader
from langchain.embeddings import OpenAIEmbeddings, OllamaEmbeddings
from langchain.llms import Ollama, BaseLLM
from langchain.schema import Document, Generation, LLMResult
from langchain.vectorstores import Chroma
from langchain_chroma import Chroma
from langchain_community.llms import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from pathlib import Path
from tqdm import tqdm
from glob import glob
import unidecode


In [4]:
import re

In [5]:

def normalize(text):
    return unidecode.unidecode(text.lower().strip())


def clean_summary_titles(summary_titles):
    cleaned = []
    for title in summary_titles:
        stripped = title.strip()
        if not stripped:
            continue  # Titre vide
        if re.fullmatch(r"[_\-–—=~\.]{3,}", stripped):
            continue  
        cleaned.append(title)
    return cleaned

def split_text_with_titles(text, summary):
    """
    Découpe le texte en segments basés sur les titres du sommaire, après normalisation,
    en respectant strictement leur ordre d'apparition attendu.
    Retourne une liste de (titre_fusionné, texte_segmenté).
    """
    split_texts = []
    merged_titles = []

    positions = []

    normalized_text = normalize(text)
    lower_text = text.lower()

    position_courante = 0

    for sentence in summary:
        norm_sentence = normalize(sentence)
        pos_norm = normalized_text.find(norm_sentence, position_courante)
        if pos_norm != -1:
            pos_real = lower_text.find(sentence.lower(), position_courante)
            if pos_real != -1:
                positions.append(pos_real)
                position_courante = pos_real + 1

    if not positions:
        # Pas de découpage, titre "Document complet"
        return [("Document complet", text)]

    positions = sorted(set(positions))
    positions.insert(0, 0)
    positions.append(len(text))

    i = 0
    while i < len(positions) - 1:
        start = positions[i]
        end = positions[i + 1]

        # Calculer les titres concernés par ce segment
        # Quand i=0, aucun titre, on peut mettre "Intro" ou vide
        if i == 0:
            titles_group = []
        else:
            # i-1 correspond à l'index du titre dans summary (car positions contient un 0 en début)
            titles_group = [summary[i-1]]

        # Vérification de fusion avec le segment suivant
        if (end - start) < 150 and i + 2 < len(positions):
            # Fusionner textes
            next_end = positions[i + 2]

            # Fusionner titres associés aux 2 segments
            if i == 0:
                titles_group = []
            else:
                # titres des deux segments fusionnés
                titles_group = [summary[i-1], summary[i]]

            merged_title = " - ".join(titles_group)
            split_texts.append(text[start:next_end].strip())
            merged_titles.append(merged_title)
            i += 2
        else:
            # Pas de fusion, un seul titre ou intro
            if i == 0:
                merged_titles.append("Introduction")
            else:
                merged_titles.append(summary[i-1])

            split_texts.append(text[start:end].strip())
            i += 1

    return list(zip(merged_titles, split_texts))

In [6]:
df = pd.read_parquet("data/echantillon_1000_hs_2024_TOC.parquet")
df = df.rename(columns={"numdossier_new":"numdossier"})
df= df.set_index("numdossier")

In [7]:
df["extracted_summary"] =  df.apply(
    lambda row: clean_summary_titles(row["extracted_summary"]),
    axis=1
)

In [8]:
df["section_dict"] = df.apply(
    lambda row: split_text_with_titles(row["accorddocx"], row["extracted_summary"]),
    axis=1
)

In [9]:

# Extraire les valeurs de la ligne avec l'index "T07524067789"
row = df.loc["T05724061388"]

# Initialiser df_test avec une seule ligne
df_test = pd.DataFrame({
    "accorddocx": [row["accorddocx"]],
    "extracted_summary": [row["extracted_summary"]],
    "section_dict": [row["section_dict"]]
}, index=["T05724061388"])


In [10]:
text = df_test["accorddocx"].iloc[0]
summary = df_test["extracted_summary"].iloc[0]
list_decoupage = df_test["section_dict"].iloc[0]

In [11]:
list_decoupage

[('Introduction',
  "Entre :\n\nLa Société AQUA DULCE, SAS inscrite au R.C.S de Nancy sous le n° 887\xa0641\xa0124 00011, et dont le siège est situé AVENUE DES SAULNIERS, 57170 CHATEAU-SALINS, représentée par Monsieur …, agissant en qualité de Président, dûment habilitée aux fins des présentes.\n\nD’une part,\n\nCi-après désignée « la Société »,\n\n\nEt :\n\nLes salariés de la Société AQUA DULCE,\nAuxquels un exemplaire de l’accord a été remis le 12/04/2024\nL’ayant approuvé, à la majorité des deux tiers, le 29/04/2024\n(Procès-verbal du référendum joint au présent accord)\n\nD'autre part,\n\nIl a été conclu le présent accord d’entreprise en application des dispositions des articles L. 2232-21 et suivants du Code du travail."),
 ('Préambule',
  'Préambule\n\nEn application des articles L. 2232-21 et suivants du Code du travail, les Parties ont souhaité conclure un accord visant à définir certains points relatifs à l’organisation du temps de travail en vigueur dans l’entreprise. \n\nLes

In [12]:
if (torch.cuda.is_available()):
    DEVICE="cuda"
else:
    DEVICE="cpu"
    
model_kwargs = {'device': "cuda"}
MODEL_NAME_EMBEDDER="BAAI/bge-m3"

embedder = HuggingFaceEmbeddings(
    model_name=MODEL_NAME_EMBEDDER, 
    model_kwargs=model_kwargs,
    show_progress=False
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [13]:
vector_store = Chroma(embedding_function=embedder,
                      persist_directory="./chroma_db_article"  #répertoire pour stocker les données
)

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [14]:
def add_chunks_to_vectorstore(df, vector_store):
    for index, row in tqdm(df.iterrows(), total=len(df)):
        numdossier = str(index)
        section_dict = row['section_dict']
        documents = []
        i = 0
        for i, (title,chunk) in enumerate(section_dict): 
            chunk = str(chunk)
            title = str(title)
            metadata = {
                "id": f"{numdossier}_{i}",
                "numdossier": numdossier, 
                 "title" : title
            }
            doc = Document(page_content=chunk, metadata=metadata)
            documents.append(doc)
            i += 1
        if documents:
            vector_store.add_documents(documents)



In [ ]:
add_chunks_to_vectorstore(df,vector_store)


  2%|▏         | 23/1000 [00:54<51:19,  3.15s/it]  

In [ ]:
!mc cp -r chroma_db_article s3/$VAULT_TOP_DIR/Rania/Accords